In [2]:
import cv2
import numpy as np

source = cv2.VideoCapture(r"C:\Users\THARINDU\Desktop\project\Image Processing\day 3\Samples\road_drive.avi")

kernel = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], np.float32)

while True:
    ret, img = source.read()
    if not ret:
        print("End of video or cannot read the frame.")
        break

    h, w = img.shape[:2]
    frameCx = w // 2

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    conv = cv2.filter2D(gray, -1, kernel)
    ret_thresh, thresh = cv2.threshold(conv, 100, 255, 0)

    thresh[0:120, :] = 0  # black upper area

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:
            cv2.drawContours(img, [cnt], -1, (0, 255, 255), 3)

            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = box.astype(int)
            cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

            cv2.line(img, (frameCx, 0), (frameCx, h), (0, 255, 0), 2)

            M = cv2.moments(cnt)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])

                if cx < frameCx:
                    distance = frameCx - cx
                    cv2.putText(img, "Distance: " + str(distance), (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                    cv2.line(img, (cx, cy), (frameCx, cy), (255, 0, 0), 2)

    cv2.imshow('LIVE', img)
    cv2.imshow('conv', conv)
    cv2.imshow('thresh', thresh)

    key = cv2.waitKey(30)  # 30ms delay is better for videos
    if key == 27:
        break

source.release()
cv2.destroyAllWindows()
